In [ ]:
import kfp
from kfp import dsl
from kubernetes.client.models import V1EnvVar, V1EnvVarSource, V1SecretKeySelector
from istio_auth_with_client import kfp_client

In [ ]:
@dsl.pipeline(
    name='pipeline_minio',
    description = 'pipeline_minio'
)

def pipeline_minio():
    secret_name = "kfp-aws-secret"
    s3_endpoint = "minio-service.kubelfow.svc.cluster.local:9000"
    minio_endpoint = "http://" + s3_endpoint
    minio_region = "us-east-1"
    
    dsl.ContainerOp(
        name='mnist-s3',
        image='블로그 저자의 개인 이미지라 접근 불가',
        #내가 하고 싶은건 이거야 이거 조립할때 여기에 minio 를 마운트 해서 main.py를 바꾸고 싶어.
        arguments=['--model', 's3://mlpipeline']
    )
    . add_env_variable(V1EnvVar(name="S3_ENDPOINT", value=s3_endpoint))

이런 느낌으로 쭉 가는데 나는 이것보다

- mc config host add minio http://localhost:9000 minio minio123  

- https://www.lesstif.com/system-admin/minio-client-76709916.html  

- mc config host add localminio 대체할_URL ACCESSKEY SECRETKEY
(아마 url엔 s3.amazonaws.com이거가 들어가면 될거 같은데)

- mc config host ls 치면 나옴

- 이런 식으로 여러 호스트를 준비 해뒀다가 필요에 따라 마운트하는게 정답이라고 생각함.

- https://min.io/docs/minio/linux/reference/minio-mc.html
- 공홈엔 이렇게 적혀 있음
- mc alias set myS3 https://s3.amazon.com/endpoint ACCESS_KEY SECRET KEY

- 쭉 보니까 jupyter에서 코드짜고 작업하다가 pvc를 volumeop로 만들어서 main.py 위치에 마운트 시키면 될듯?
- 아니네... 주피터에서 바로 줄 수가 없구나... 그래서 minio로 하려고 한건데
- 이거보다 kubeflow fairing으로 dockerize 시키는게 더 효율적인가?

이렇게 등록해둔 호스트를 가지고

이거는 pvc를 그냥 k8s 차원에서 연결하는거고

In [ ]:
dvop = dsl.VolumeOp(name="create_pvc",
                    resource_name="my-pvc-2",
                    size="5Gi",
                    modes=dsl.VOLUME_MODE_RWO)

minio 쓰는건

In [ ]:
train = dsl.ContainerOp(
    name='trainmodel',
    image = '이미지') \
    .add_env_variable(k8s_client.V1EnvVar(name='MINIO_URL', value=내용)) \ #'S3_URL
    .add_env_variable(k8s_client.V1EnvVar(name='access')
                                                  'secret'

하면 될거 같음
근데 이거 넣는건 넣는건데 어디 위치에 마운트 되는거지? 로컬에서 한번 써보느넥 아니면 감 잡기 쉽지 않을거 같은데?

loop로 미리 셋팅해두면 각 stage에서 쉬워질수 이다고 적혀있음.

for each in pipeline:
    each.add_env_variable(k8s_client.V1EnvVar(name='MINIO_URL', value=내용)) 
    이런식인듯

In [ ]:
겉에서 연결하는 것 이런건 좀 알겠는데

쓰면 마운트가 어떻게 되는건지 이런것들은 좀 써봐야겠음.
fs에 마운트 되는건지 함수로 호출하는건지 등등

In [ ]:
어느 디렉토리에 어떻게 마운트 되었는지 표기가 안되어있어서... 코드에서 직접 연결하는건가?

In [ ]:
import kfp
from kfp import dsl
from kubernetes.client.models import V1EnvVar, V1EnvVarSource, V1SecretKeySelector
from istio_auth_with_client import kfp_client

In [ ]:
import kfp
from kfp import dsl
from kfp import aws

from kubernetes.client.models import V1EnvVar


@dsl.pipeline(
    name='spark-tf_pipeline',
    description = 'pipeline with s3'
)
def pipeline_use_aws_secret():
    preprocessing = dsl.ContainerOp(
        name = "spark_job",
        image = "apache/spark:3.4.1-python3" #gcr.io/spark-operator/spark-py:v3.4.1
    )
        .add_env_variable(aws.use_aws_secret("aws-secret",#secret_name,
                                             aws_access_key_id_name = 'AWS_ACCESS_KEY_ID',
                                             aws_secret_access_key_name = 'AWS_SECRET_ACCESS_KEY'))
        .add_env_variable(V1EnvVar(name='AWS_REGION', value = 'us-east1'))

    training = dsl.ContainerOp(
        name = 'distributed_tf_job',
        image = 'tensorflow/tensorflow:2.12.0'
    )
        .add_env_variable(V1EnvVar(name='MINIO_URL', value='http://minio-service.kubeflow.svc.cluster.local:9000'))
        .add_env_variable(V1EnvVar(name='MINIO_KEY', value='minio'))
        .add_env_variable(V1EnvVar(name='MINIO_SECRET', value='minio123'))



In [ ]:
training = dsl.ContainerOp(
    name = "training_process",
    image = "tensorflow/tensorflow:2.12.0"
)

In [ ]:
pv 달고 다운로드 + 작업

다운로드랑 작업이랑 나눠야 되나?

pv에 다운로드 + 
